In [1]:
from google.colab import drive

drive.mount('/content/gdrive')
root_path = 'gdrive/My Drive/'

Mounted at /content/gdrive


# **Training_Corpus**


In [4]:
def load_dataset(csv_file):
    return pd.read_csv(csv_file)


In [5]:
import pandas as pd

def get_weight_schemes(author_info):
    user_validlevel = author_info['validation_level']
    user_rank = author_info['rank_level']
    if user_rank == 'unk':
        user_rank = 'level_0'

    answer_profile = [
        1 if str(user_validlevel)[:2] == 'md' else 0,
        1 if int(user_rank.replace('level_', '')) >= 4 else 0,
        1 if author_info['contains_freq_ans'] == 1.0 else 0,
        1 if author_info['completeness'] == 1.0 else 0,
    ]

    w1 = -100
    if answer_profile[-1] == 1:
        first_3 = sum(answer_profile[:-1])
        if first_3 == 3:
            w1 = 1.0
        elif first_3 == 2:
            w1 = 0.9
        elif first_3 == 1:
            w1 = 0.8
        else:
            w1 = 0.7
    else:
        first_3 = sum(answer_profile[:-1])
        if first_3 == 3:
            w1 = 0.9
        elif first_3 == 2:
            w1 = 0.8
        elif first_3 == 1:
            w1 = 0.7
        else:
            w1 = 0.6

    return w1

def add_author_weights(df):
    author_weights = []

    for index, row in df.iterrows():
        author_info = {
            'validation_level': row['validation_level'],
            'rank_level': row['rank_level'],
            'contains_freq_ans': row['contains_freq_ans'],
            'completeness': row['completeness']
        }
        author_weights.append(get_weight_schemes(author_info))

    df['author_weight'] = author_weights

# Load your CSV file into a DataFrame
df = pd.read_csv('gdrive/My Drive/mediqa-m3-clinicalnlp2024/merged_profileAuthor.csv')

# Apply the function to add author weights
add_author_weights(df)

# Print the DataFrame with the added author weights
print(df)


      author_id  completeness  contains_freq_ans validation_level rank_level  \
0        U00761           0.5                0.0    md2_validated        unk   
1        U00761           0.5                0.0    md2_validated        unk   
2        U00761           0.5                1.0    md2_validated        unk   
3        U00761           0.5                1.0    md2_validated        unk   
4        U00761           0.5                1.0    md2_validated        unk   
...         ...           ...                ...              ...        ...   
20671    U20517           0.0                0.0    md1_validated    level_0   
20672    U20518           0.0                0.0    md1_validated    level_0   
20673    U20519           0.0                0.0    md1_validated    level_0   
20674    U20520           0.0                0.0    md1_validated    level_0   
20675    U20521           0.0                0.0    md1_validated    level_0   

       author_weight  
0               

In [6]:
# Save the modified DataFrame to a new CSV file
df.to_csv('gdrive/My Drive/mediqa-m3-clinicalnlp2024/weight_profileAuthor.csv', index=False)


In [ ]:
gdrive/My Drive/mediqa-m3-clinicalnlp2024/train.json

In [8]:
import pandas as pd
import json

# Load JSON data
with open('gdrive/My Drive/mediqa-m3-clinicalnlp2024/train.json', 'r') as f:
    data = json.load(f)

# Convert JSON data to DataFrame
df = pd.json_normalize(data)

# Print DataFrame
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 842 entries, 0 to 841
Data columns (total 10 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   encounter_id      842 non-null    object
 1   author_id         842 non-null    object
 2   image_ids         842 non-null    object
 3   query_title_zh    842 non-null    object
 4   query_content_zh  842 non-null    object
 5   responses         842 non-null    object
 6   query_title_en    842 non-null    object
 7   query_content_en  842 non-null    object
 8   query_title_es    842 non-null    object
 9   query_content_es  842 non-null    object
dtypes: object(10)
memory usage: 65.9+ KB
None


In [9]:
df['responses'][0]

[{'author_id': 'U00217',
  'content_zh': '银屑病，似与胸腔积液没有关系',
  'content_en': 'Psoriasis seems to have no relation to pleural effusion.',
  'content_es': 'La psoriasis parece no tener relación con el derrame pleural.'},
 {'author_id': 'U00214',
  'content_zh': '典型的银屑病',
  'content_en': 'Typical Psoriasis',
  'content_es': 'Psoriasis Típica'},
 {'author_id': 'U00088',
  'content_zh': '银屑病',
  'content_en': 'Psoriasis',
  'content_es': 'Psoriasis'},
 {'author_id': 'U00051',
  'content_zh': '典型的银屑病',
  'content_en': 'Typical Psoriasis',
  'content_es': 'Psoriasis Típica'}]

In [14]:
import json
import pandas as pd

# Load JSON data
with open('gdrive/My Drive/mediqa-m3-clinicalnlp2024/train.json', 'r') as f:
    data = json.load(f)

# Load DataFrame with author weights
# Assuming you have a DataFrame named author_weights with columns ['author_id', 'weight']
author_weights = pd.read_csv('gdrive/My Drive/mediqa-m3-clinicalnlp2024/weight_profileAuthor.csv')

# Create a dictionary to store author weights
author_weight_dict = author_weights.set_index('author_id')['author_weight'].to_dict()

# Create a DataFrame to store selected responses
selected_data = []

# Iterate over each item in the JSON data
for item in data:
    encounter_id = item['encounter_id']
    image_ids = item['image_ids']
    query_title_en = item['query_title_en']
    query_content_en = item['query_content_en']
    query_title_zh = item['query_title_zh']
    query_content_zh = item['query_content_zh']
    query_title_es = item['query_title_es']
    query_content_es = item['query_content_es']
    responses = item['responses']

    # Initialize variables to store the highest weight and corresponding response
    max_weight = float('-inf')
    max_response = None

    # Iterate over each response in the item
    for response in responses:
        author_id = response['author_id']
        content_en = response['content_en']
        content_zh = response['content_zh']
        content_es = response['content_es']

        # Check if the author has a weight in the author_weight_dict
        if author_id in author_weight_dict:
            # Compare the weight of the current author with the max_weight
            if author_weight_dict[author_id] > max_weight:
                max_weight = author_weight_dict[author_id]
                max_response = {
                    'author_id': author_id,
                    'content_en': content_en,
                    'content_zh': content_zh,
                    'content_es': content_es
                }

    # Add the encounter_id and selected response to the selected_data list
    selected_data.append({
        'encounter_id': encounter_id,
        'image_ids': image_ids,
        'query_title_en': query_title_en,
        'query_content_en': query_content_en,
        'author_id': max_response['author_id'] if max_response else None,
        'content_en': max_response['content_en'] if max_response else None,
        'content_zh': max_response['content_zh'] if max_response else None,
        'content_es': max_response['content_es'] if max_response else None
    })

# Create DataFrame from selected_data
selected_df = pd.DataFrame(selected_data)

# Print the DataFrame
print(selected_df)


    encounter_id                                          image_ids  \
0       ENC00001   [IMG_ENC00001_00001.jpg, IMG_ENC00001_00002.jpg]   
1       ENC00002  [IMG_ENC00002_00001.jpg, IMG_ENC00002_00002.jp...   
2       ENC00003  [IMG_ENC00003_00001.jpg, IMG_ENC00003_00002.jp...   
3       ENC00004   [IMG_ENC00004_00001.jpg, IMG_ENC00004_00002.jpg]   
4       ENC00005                           [IMG_ENC00005_00001.jpg]   
..           ...                                                ...   
837     ENC00847                           [IMG_ENC00847_00001.jpg]   
838     ENC00848  [IMG_ENC00848_00001.jpg, IMG_ENC00848_00002.jp...   
839     ENC00849   [IMG_ENC00849_00001.jpg, IMG_ENC00849_00002.jpg]   
840     ENC00850                           [IMG_ENC00850_00001.jpg]   
841     ENC00851  [IMG_ENC00851_00001.jpg, IMG_ENC00851_00002.jp...   

                                        query_title_en  \
0                 Pleural effusion accompanied by rash   
1             What is on the bo

In [15]:
selected_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 842 entries, 0 to 841
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   encounter_id      842 non-null    object
 1   image_ids         842 non-null    object
 2   query_title_en    842 non-null    object
 3   query_content_en  842 non-null    object
 4   author_id         842 non-null    object
 5   content_en        842 non-null    object
 6   content_zh        842 non-null    object
 7   content_es        842 non-null    object
dtypes: object(8)
memory usage: 52.8+ KB


In [12]:
selected_df['content_zh'][0]

'是继发性白斑。多为暂时的，一般不需治疗，经半年至一年左右，可自行恢复。'

In [13]:
selected_df1 = selected_df

In [16]:
import pandas as pd

# Assuming df1 and df2 are your DataFrames with the same columns
# Concatenate the two DataFrames
merged_df = pd.concat([selected_df1, selected_df], ignore_index=True)

# Print the merged DataFrame
print(merged_df)


    encounter_id                                          image_ids  \
0       ENC00852   [IMG_ENC00852_00001.jpg, IMG_ENC00852_00002.jpg]   
1       ENC00853   [IMG_ENC00853_00001.jpg, IMG_ENC00853_00002.jpg]   
2       ENC00854  [IMG_ENC00854_00001.jpg, IMG_ENC00854_00002.jp...   
3       ENC00855   [IMG_ENC00855_00001.jpg, IMG_ENC00855_00002.jpg]   
4       ENC00856                           [IMG_ENC00856_00001.jpg]   
..           ...                                                ...   
893     ENC00847                           [IMG_ENC00847_00001.jpg]   
894     ENC00848  [IMG_ENC00848_00001.jpg, IMG_ENC00848_00002.jp...   
895     ENC00849   [IMG_ENC00849_00001.jpg, IMG_ENC00849_00002.jpg]   
896     ENC00850                           [IMG_ENC00850_00001.jpg]   
897     ENC00851  [IMG_ENC00851_00001.jpg, IMG_ENC00851_00002.jp...   

                                        query_title_en  \
0               Is this Vitiligo?  Please see picture.   
1    Please help take a look, w

In [17]:
merged_df.to_csv('gdrive/My Drive/mediqa-m3-clinicalnlp2024/trainingCorpusM3G.csv', index=False)
